In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import cv2 
from PIL import Image
import glob
from skimage import feature
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import pickle
import pywt
import re
import copy
import random


import warnings
warnings.filterwarnings('ignore')

In [14]:
def detect_contour(image):
    obj_dict = {}
    retval, bw = cv2.threshold(image, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    detect_count = 0

    for i in range(0, len(contours)):
        area = cv2.contourArea(contours[i])
        
        if area < 1e2 or 1e5 < area:
            continue
            
        if len(contours[i]) > 0:
            rect = contours[i]
            x, y, w, h = cv2.boundingRect(rect)
            
            angle_data = cv2.minAreaRect(contours[i])
            angle = angle_data[2]
            point = [y, y+h, x, x+w, angle]
            w_h = w + h
            obj_dict[w_h] = point 
    return obj_dict[max(obj_dict)]

In [3]:
def burst_wave(img, wavelet_list, level, IMAGE_WIDTH=640, IMAGE_HEIGHT=480, M_WAVELET="db1", mode="sym"):
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    data = img.astype(np.float64)
    coeffs = pywt.wavedec2(data, M_WAVELET, level=level, mode=mode)

    wavelet_data = np.array([])
    for i, data in enumerate(coeffs):
        data = np.ravel(data)
        wavelet_data = np.append(wavelet_data, data)
    wavelet_list.append(wavelet_data)
    
    return wavelet_list

In [28]:
color_path = glob.glob('image_folder/*/*color*')
depth_path = []

random.seed(0)
random.shuffle(color_path)
for path in color_path:
    new_path = path.replace('color', 'depth')
    depth_path.append(new_path)
    
train_color = color_path[:70]
train_depth = depth_path[:70]

test_color = color_path[70:]
test_depth = depth_path[70:]

In [29]:
def make_wave_feature(color_paths, depth_paths, mode):
    wavelet_list = []
    name_list = []
    LEVEL = 5
    for color_path, depth_path in zip(color_paths, depth_paths):
        depth_image = cv2.imread(depth_path, 0)
        color_image = cv2.imread(color_path, 0)

        # y_trainの作成
        slash_index = color_path[13:].find('/')
        name_list.append(color_path[13:13+slash_index])

        image_list = []
        depth_copy = copy.deepcopy(depth_image)
        color_copy = copy.deepcopy(color_image)

        for col in depth_image:
            for one in col:
                if one > 1000:
                    image_list.append(0)
                else:
                    image_list.append(one)

        for y, col in enumerate(depth_image):
            for x, pixel in enumerate(col):
                if pixel > 0 and pixel < 390: 
                    depth_image[y][x] = 255
                else:
                    depth_image[y][x] = 0
        image = np.array(depth_image, dtype=np.uint8)

        y1, y2, x1, x2, angle = detect_contour(image)
        img = color_image[y1:y2, x1:x2]
        depth_cut = depth_copy[y1:y2, x1:x2]

        wavelet_list = burst_wave(img, wavelet_list, level=5, IMAGE_WIDTH=640, IMAGE_HEIGHT=480, M_WAVELET="db1", mode="sym")

    X_train = np.array(wavelet_list)
    
    if mode == 'train':
        y_dict = {}
        le = LabelEncoder()
        y_train = le.fit_transform(name_list)
        decoded = le.inverse_transform(y_train)

        for name, name_id in zip(decoded, y_train):
            y_dict[name_id] = name
        return X_train, y_train, y_dict
    else:
        return X_train
    
    return train_wave

In [30]:
X_train, y_train, y_dict = make_wave_feature(train_color, train_depth, 'train')
X_test = make_wave_feature(test_color, test_depth, 'train')

In [35]:
X_train.shape

(70, 307200)

In [32]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [36]:
y_pred = knn.predict(X_test[0])
for index in y_pred:
    print(y_dict[index])

iris
apple
curry
curry
kurumiB


In [37]:
test_color

['image_folder/iris/iris_color_1.jpg',
 'image_folder/apple/apple_color_0_221.jpg',
 'image_folder/curry/curry_color_0_868.jpg',
 'image_folder/kurumiA/kurumiA_color_0.jpg',
 'image_folder/kurumiA/kurumiA_color_0_346.jpg']